In [1]:
# パッケージをロード
library(wooldridge)

### 大問1


In [2]:
# charityデータをロード
data("charity")

#### 1.

In [3]:
# モデル1
model1 <- lm(gift ~ mailsyear, data = charity)

# モデル2
model2 <- lm(gift ~ mailsyear + giftlast + propresp, data = charity)

# モデル3
model3 <- lm(gift ~ mailsyear + giftlast + propresp + avggift, data = charity)

In [4]:
summary(model1)["coefficients"]
summary(model2)["coefficients"]
summary(model3)["coefficients"]

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),2.014080,0.7394696,2.723682,6.482158e-03
mailsyear,2.649546,0.3430598,7.723279,1.403549e-14


,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),-4.551518496,0.803033625,-5.667905,1.540942e-08
mailsyear,2.166259248,0.331927051,6.526311,7.529112e-11
giftlast,0.005926546,0.001432357,4.137617,3.576994e-05
propresp,15.358605277,0.874539404,17.561936,1.022183e-66


,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),-7.3277630,0.75821999,-9.664429,7.146568e-22
mailsyear,1.2011684,0.31241799,3.844748,1.224363e-04
giftlast,-0.2608573,0.01075649,-24.251166,8.140719e-122
propresp,16.2046425,0.81752921,19.821484,1.030457e-83
avggift,0.5269471,0.02108106,24.996228,8.391669e-129


#### 2.

In [5]:
results <- data.frame(Model = c("Model 1", "Model 2", "Model 3"),
                      R_squared = c(summary(model1)$r.squared, summary(model2)$r.squared, summary(model3)$r.squared),
                      Sample_Size = c(nrow(charity), nrow(charity), nrow(charity)))

results

Model,R_squared,Sample_Size
<chr>,<dbl>,<int>
Model 1,0.01378962,4268
Model 2,0.08335990,4268
Model 3,0.20053447,4268


説明変数が増えるにつれ, 決定係数が大きくなっている.  
これは, 説明変数の増加によってモデルがより詳細に説明する能力を得るためである. ただし, むやみに説明変数を増加させればよいというわけではなく, 過学習の問題を考慮する必要がある.

#### 3.

$mailsyear$の係数は2.17であり, 年間で送られるメールの数が1増えると、寄付額が2.17程度増えると解釈できる.  
また, この係数は単回帰モデルの場合(2.65)より小さくなっている. これは他の変数の影響を考慮した結果、$mailsyear$の影響が相対的に小さくなったためと考えられる.

#### 4.

$propresp$は, 送られたメールに対する応答の割合であり, モデル2ではその係数が15.36である.  
つまり, 応答の割合が1%増えると, 寄付額が15.36増えると解釈できる.

#### 5.

モデル2とモデル3で, $giftlast$の係数が正から負に変化している.  
これは, $giftlast$と$avggift$の相関(0.99)が強く, 多重共線性を起こしているためであると推測できる.

In [6]:
# giftlastとavggiftの相関係数
correlation <- cor(charity$avggift, charity$giftlast)
correlation
# 0.992113087513734

[1] 0.9921131

### 大問2

In [7]:
# discrimデータをロード
data("discrim")

#### 1.

In [8]:
model <- lm(log(psoda) ~ prpblck + log(income) + prppov, data = discrim)
summary(model)["coefficients"]

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),-1.46333208,0.29371104,-4.982217,9.400188e-07
prpblck,0.07280726,0.03067561,2.373458,1.809760e-02
log(income),0.13695523,0.02675544,5.118780,4.802041e-07
prppov,0.38035969,0.13279029,2.864364,4.400360e-03


#### 2.

model:　$ \log{(psoda)} = \beta_0 + \beta_1prpblck + \beta_2\log{(income)} + \beta_3 prppov + u$

In [9]:
beta_hat_1 <- summary(model)$coefficients["prpblck", "Estimate"]
beta_hat_2 <- summary(model)$coefficients["log(income)", "Estimate"]
beta_hat_3 <- summary(model)$coefficients["prppov", "Estimate"]

model_a:　$ \hat{\log{(income)}} = \hat{\alpha_0} + \hat{\alpha_1} prpblck $  

model_b:　$ \hat{prppov} = \hat{\gamma_0} + \hat{\gamma_1} prpblck $  

model_c:　$\tilde{\log{(psoda)}} = \tilde{\beta_0} + \tilde{\beta_1} prpblck $   

In [10]:
model_a <- lm(log(income) ~ prpblck, data = discrim)
alpha_hat_1 <- summary(model_a)$coefficients["prpblck", "Estimate"]

model_b <- lm(prppov ~ prpblck, data = discrim)
gamma_hat_1 <- summary(model_b)$coefficients["prpblck", "Estimate"]

model_c <- lm(log(psoda) ~ prpblck, data = discrim)

$ \tilde{\beta_1} \approx \hat{\beta_1} + \hat{\beta_2}\hat{\alpha_1} + \hat{\beta_3}\hat{\gamma_1} $　　を確認

In [11]:
beta_tilde_1 = summary(model_c)$coefficients["prpblck", "Estimate"]
beta_tilde_1
# 0.0624662254037403

[1] 0.06246623

In [12]:
beta_hat_1 + (beta_hat_2 * alpha_hat_1) + (beta_hat_3 * gamma_hat_1)
# 0.0623943196709311

[1] 0.06239432

回帰解剖アプローチによる$\beta_1$の推定量は0.062となった.  
これは他の変数$income$や$prppov$の影響を取り除いた, $\log{(psoda)}$ に対する$prpblck$の純粋な回帰係数といえる.

#### 3.

In [13]:
model <- lm(log(psoda) ~ prpblck + log(income) + prppov + log(hseval), data = discrim)
summary(model)["coefficients"]

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),-0.84151459,0.29243178,-2.877644,4.223540e-03
prpblck,0.09755020,0.02926073,3.333826,9.373070e-04
log(income),-0.05299049,0.03752607,-1.412098,1.587065e-01
prppov,0.05212273,0.13449918,0.387532,6.985706e-01
log(hseval),0.12130567,0.01768407,6.859601,2.668125e-11


$\log{(seval)}$の回帰係数は0.121であり, これは$seval$が1%増加したときに, $psoda$が1%増加することを示している.  
また, p値は$2.66\times 10^{-11}$であり, 両側5%で有意である.

#### 4.

$\log{(income)}$と$prppov$のp値はそれぞれ$1.59\times10^{-1}$, $6.99\times10^{-1}$ であり,  
両側5%で有意ではない.

#### 5.

推定したモデルの結果を用いて価格差別があるかどうかを論じるために、prpblck（黒人の割合）の係数に着目する. この係数は正, そしてp値は$9.37\times10^{-4}$であり, 統計的に有意である. ゆえに黒人の割合が高い地域でソーダの価格が高くなるという価格差別の傾向があると解釈できるが, 欠落変数が存在する可能性がある. 

欠落変数の例: 各地域における夏場の平均気温  
夏場の平均気温が高い地域ではソーダの需要が大きく, 価格が高くなると予測できる. そして, その地域における黒人割合が高ければ, 黒人割合が高い$\Rightarrow$ソーダの価格が高いという疑似相関が発生する可能性がある.